In [1]:
import os
import multiprocessing
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Data Image

In [3]:
# Define the training and validation base directories
train_dir = "/content/drive/MyDrive/Colab Notebooks/model_test_2/Dataset/Train"
validation_dir = "/content/drive/MyDrive/Colab Notebooks/model_test_2/Dataset/Valid"

def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (400, 400))
    image = tf.cast(image, tf.float32)
    image = image / 255.0
    return image, label

# Hyperparameter

In [4]:
num_classes = 15
input_shape = (400, 400, 3)
Batch_size = 128
epochs = 15

# Generator

In [5]:
def train_val_generators(train_dir, validation_dir):
#TRAIN   
  train_datagen = ImageDataGenerator(rescale=1./255,
                                     rotation_range=40,
                                     width_shift_range = 0.2,
                                     height_shift_range = 0.2,
                                     shear_range = 0.2,
                                     zoom_range = 0.2,
                                     horizontal_flip = True,
                                     fill_mode = "nearest")

  train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                      batch_size=Batch_size, 
                                                      class_mode="categorical",
                                                      target_size=(400, 400))

#VALIDATION
  validation_datagen = ImageDataGenerator(rescale=1/255)


  validation_generator = validation_datagen.flow_from_directory(directory=validation_dir,
                                                                batch_size=Batch_size, 
                                                                class_mode="categorical",
                                                                target_size=(400, 400))
  return train_generator, validation_generator

In [6]:
train_generator, validation_generator = train_val_generators(train_dir, validation_dir)

Found 7146 images belonging to 15 classes.
Found 894 images belonging to 15 classes.


# Extract

In [7]:
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_signature=(
        tf.TensorSpec(shape=(), dtype=tf.string),
        tf.TensorSpec(shape=(), dtype=tf.float32)
    )
)

validation_dataset = tf.data.Dataset.from_generator(
    lambda: validation_generator,
    output_signature=(
        tf.TensorSpec(shape=(), dtype=tf.string),
        tf.TensorSpec(shape=(), dtype=tf.float32)
    )
)

# Transform

In [8]:
cores = multiprocessing.cpu_count()

train_dataset = train_dataset.map(load_and_preprocess_image, num_parallel_calls=cores)
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(1024)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)


validation_dataset = validation_dataset.map(load_and_preprocess_image, num_parallel_calls=cores)
validation_dataset = validation_dataset.cache()
validation_dataset = validation_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [9]:
# Transfer Learning
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [12]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

    for layer in base_model.layers:
      layer.trainable = False

    #tambah layer 
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    
    
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator,
                        steps_per_epoch=train_generator.n // train_generator.batch_size,
                        epochs=epochs,
                        validation_data=validation_generator,
                        validation_steps=validation_generator.n // validation_generator.batch_size)

Epoch 1/15
55/55 [==============================] - 1686s 30s/step - loss: 2.1603 - accuracy: 0.2968 - val_loss: 1.2483 - val_accuracy: 0.6315
Epoch 2/15
55/55 [==============================] - 459s 8s/step - loss: 1.2874 - accuracy: 0.5644 - val_loss: 0.6818 - val_accuracy: 0.8125
Epoch 3/15
55/55 [==============================] - 456s 8s/step - loss: 0.9559 - accuracy: 0.6654 - val_loss: 0.5542 - val_accuracy: 0.8307
Epoch 4/15
55/55 [==============================] - 455s 8s/step - loss: 0.8297 - accuracy: 0.7087 - val_loss: 0.4429 - val_accuracy: 0.8724
Epoch 5/15
55/55 [==============================] - 449s 8s/step - loss: 0.7515 - accuracy: 0.7368 - val_loss: 0.4242 - val_accuracy: 0.8685
Epoch 6/15
55/55 [==============================] - 446s 8s/step - loss: 0.6868 - accuracy: 0.7551 - val_loss: 0.3335 - val_accuracy: 0.9023
Epoch 7/15
55/55 [==============================] - 448s 8s/step - loss: 0.6365 - accuracy: 0.7689 - val_loss: 0.3214 - val_accuracy: 0.9049
Epoch 8/15
